# Data Extraction and Merging:
### In this notebook, we extract the required data from 6 datasets and then merge them together to be used for Machine Learning model buiding later (see R notebook).

First, we aim to extract the monthly inflation rate of India from the OECD dataset [1] which has monthly inflation for all countries.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
infl_df = pd.read_csv("inflation_monthly.csv")
infl_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUT,CPI,TOT,AGRWTH,M,2000-01,1.149740,NaN
1,AUT,CPI,TOT,AGRWTH,M,2000-02,1.639344,NaN
2,AUT,CPI,TOT,AGRWTH,M,2000-03,1.939891,NaN
3,AUT,CPI,TOT,AGRWTH,M,2000-04,1.857923,NaN
4,AUT,CPI,TOT,AGRWTH,M,2000-05,1.745296,NaN


Now, we filter the dataframe such that the "LOCATION" column only the values "IND"

In [3]:
filt = (infl_df["LOCATION"] == "IND")
infl_df[filt].head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
8030,IND,CPI,TOT,AGRWTH,M,2000-01,2.619048,NaN
8031,IND,CPI,TOT,AGRWTH,M,2000-02,3.614458,NaN
8032,IND,CPI,TOT,AGRWTH,M,2000-03,4.830918,NaN
8033,IND,CPI,TOT,AGRWTH,M,2000-04,5.542169,NaN
8034,IND,CPI,TOT,AGRWTH,M,2000-05,5.011933,NaN


From the entire dataset, we only require the time and the value. The values will be stores as a dataframe object called "final"

In [4]:
final = infl_df[filt].loc[:,["TIME", "Value"]]
final.count()

TIME     250
Value    250
dtype: int64

We rename the "TIME' column into lowercase and set it new indexes for the dataframe

In [5]:
final.rename(columns={"TIME":"Time"}, inplace=True)

In [6]:
newin = list(range(0,250))
final.set_index(pd.Index(newin), inplace=True)

In [7]:
final

,Time,Value
0,2000-01,2.619048
1,2000-02,3.614458
2,2000-03,4.830918
3,2000-04,5.542169
4,2000-05,5.011933
...,...,...
245,2020-06,5.063291
246,2020-07,5.329154
247,2020-08,5.625000
248,2020-09,5.636574


All the remaining features will be seprataley extracted from their respective CSV files, then appended to this `final` dataframe.  
Now, we work with the USD to INR monthly exchange rate dataset [2] from FRED (Federal Reserve Economic Data) by performing necessary modifications and then appending it to the final data frame.

In [8]:
exc_df = pd.read_csv("exchange_rate_monthly.csv")
exc_df

,DATE,EXINUS
0,1973-01-01,8.0041
1,1973-02-01,7.7538
2,1973-03-01,7.5465
3,1973-04-01,7.5462
4,1973-05-01,7.4927
...,...,...
571,2020-08-01,74.5657
572,2020-09-01,73.5233
573,2020-10-01,73.5648
574,2020-11-01,74.2311


In [9]:
exc_df["DATE"] = pd.to_datetime(exc_df["DATE"], format = "%Y-%m-%d")

In [10]:
datefilt = (exc_df["DATE"] >= "2000")
exc_df[datefilt]

,DATE,EXINUS
324,2000-01-01,43.5895
325,2000-02-01,43.6540
326,2000-03-01,43.6383
327,2000-04-01,43.6847
328,2000-05-01,44.0759
...,...,...
571,2020-08-01,74.5657
572,2020-09-01,73.5233
573,2020-10-01,73.5648
574,2020-11-01,74.2311


We remove the last two columns i.e., of November and December 2020 as the corresponding values for these months is missing for some of the other features.

In [11]:
exc_df2 = exc_df[datefilt].loc[324:573]
exc_df2

,DATE,EXINUS
324,2000-01-01,43.5895
325,2000-02-01,43.6540
326,2000-03-01,43.6383
327,2000-04-01,43.6847
328,2000-05-01,44.0759
...,...,...
569,2020-06-01,75.7077
570,2020-07-01,74.9286
571,2020-08-01,74.5657
572,2020-09-01,73.5233


In [12]:
exc_df2.set_index(pd.Index(newin), inplace=True)
exc_df2

,DATE,EXINUS
0,2000-01-01,43.5895
1,2000-02-01,43.6540
2,2000-03-01,43.6383
3,2000-04-01,43.6847
4,2000-05-01,44.0759
...,...,...
245,2020-06-01,75.7077
246,2020-07-01,74.9286
247,2020-08-01,74.5657
248,2020-09-01,73.5233


In [13]:
exc_df2["DATE"]

0     2000-01-01
1     2000-02-01
2     2000-03-01
3     2000-04-01
4     2000-05-01
         ...    
245   2020-06-01
246   2020-07-01
247   2020-08-01
248   2020-09-01
249   2020-10-01
Name: DATE, Length: 250, dtype: datetime64[ns]

In [14]:
exc_df2["DATE"] = exc_df2["DATE"].apply(lambda x: datetime.strftime(x, "%Y-%m"))
exc_df2

,DATE,EXINUS
0,2000-01,43.5895
1,2000-02,43.6540
2,2000-03,43.6383
3,2000-04,43.6847
4,2000-05,44.0759
...,...,...
245,2020-06,75.7077
246,2020-07,74.9286
247,2020-08,74.5657
248,2020-09,73.5233


In [15]:
exc_df2.rename(columns={"DATE":"Time", "EXINUS":"Exchange"}, inplace=True)

In [16]:
exc_df2.head()

,Time,Exchange
0,2000-01,43.5895
1,2000-02,43.6540
2,2000-03,43.6383
3,2000-04,43.6847
4,2000-05,44.0759


Clearly, we have modified the exchange rate dataframe enough that we can append it to the `final` dataframe on the basis `Time` column

In [17]:
exc_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Time      250 non-null    object 
 1   Exchange  250 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.9+ KB


In [18]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    250 non-null    object 
 1   Value   250 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.9+ KB


In [19]:
final = final.merge(exc_df2, on="Time")
final

,Time,Value,Exchange
0,2000-01,2.619048,43.5895
1,2000-02,3.614458,43.6540
2,2000-03,4.830918,43.6383
3,2000-04,5.542169,43.6847
4,2000-05,5.011933,44.0759
...,...,...,...
245,2020-06,5.063291,75.7077
246,2020-07,5.329154,74.9286
247,2020-08,5.625000,74.5657
248,2020-09,5.636574,73.5233


Now, we do the exact same operations with "ratio with exports to imports" dataset [3] also from FRED.

In [20]:
rat_df = pd.read_csv("export_to_import_monthly.csv")
rat_df

,DATE,XTEITT01INM156S
0,1990-01-01,79.847374
1,1990-02-01,73.923665
2,1990-03-01,83.165359
3,1990-04-01,74.635606
4,1990-05-01,86.613671
...,...,...
365,2020-06-01,105.855326
366,2020-07-01,86.198660
367,2020-08-01,78.095836
368,2020-09-01,87.053092


In [21]:
rat_df["DATE"] = pd.to_datetime(rat_df["DATE"], format = "%Y-%m-%d")

In [22]:
datefilt2 = (rat_df["DATE"] >= "2000")
rat_df[datefilt2]

,DATE,XTEITT01INM156S
120,2000-01-01,69.665932
121,2000-02-01,73.462223
122,2000-03-01,70.899585
123,2000-04-01,75.393458
124,2000-05-01,86.969321
...,...,...
365,2020-06-01,105.855326
366,2020-07-01,86.198660
367,2020-08-01,78.095836
368,2020-09-01,87.053092


In [23]:
rat_df = rat_df[datefilt2]
rat_df.set_index(pd.Index(newin), inplace=True)
rat_df["DATE"] = rat_df["DATE"].apply(lambda x: datetime.strftime(x, "%Y-%m"))
rat_df.rename(columns={"DATE":"Time", "XTEITT01INM156S":"Ratio_E_I"}, inplace=True)
rat_df

c:\users\najju\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\najju\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Time,Ratio_E_I
0,2000-01,69.665932
1,2000-02,73.462223
2,2000-03,70.899585
3,2000-04,75.393458
4,2000-05,86.969321
...,...,...
245,2020-06,105.855326
246,2020-07,86.198660
247,2020-08,78.095836
248,2020-09,87.053092


In [24]:
final = final.merge(rat_df, on="Time")
final

,Time,Value,Exchange,Ratio_E_I
0,2000-01,2.619048,43.5895,69.665932
1,2000-02,3.614458,43.6540,73.462223
2,2000-03,4.830918,43.6383,70.899585
3,2000-04,5.542169,43.6847,75.393458
4,2000-05,5.011933,44.0759,86.969321
...,...,...,...,...
245,2020-06,5.063291,75.7077,105.855326
246,2020-07,5.329154,74.9286,86.198660
247,2020-08,5.625000,74.5657,78.095836
248,2020-09,5.636574,73.5233,87.053092


We need to the same procedure for monthly interest rates [4] and monthly money supply [5]. Both datasets are from FRED as well.

In [25]:
int_df = pd.read_csv("interest_monthly.csv")
int_df

,DATE,INTDSRINM193N
0,01-01-68,6.00
1,01-02-68,6.00
2,01-03-68,5.00
3,01-04-68,5.00
4,01-05-68,5.00
...,...,...
629,01-06-20,4.25
630,01-07-20,4.25
631,01-08-20,4.25
632,01-09-20,4.25


In [26]:
int_df["DATE"] = pd.to_datetime(int_df["DATE"], format = "%d-%m-%y")

In [27]:
datefilt3 = (int_df["DATE"] >= "2000") & (int_df["DATE"] < "2021") #as 68 to 99 are interpreted as 2068 to 2099
int_df[datefilt3]

,DATE,INTDSRINM193N
384,2000-01-01,8.00
385,2000-02-01,8.00
386,2000-03-01,8.00
387,2000-04-01,7.00
388,2000-05-01,7.00
...,...,...
629,2020-06-01,4.25
630,2020-07-01,4.25
631,2020-08-01,4.25
632,2020-09-01,4.25


In [28]:
int_df = int_df[datefilt3]
int_df.set_index(pd.Index(newin), inplace=True)
int_df["DATE"] = int_df["DATE"].apply(lambda x: datetime.strftime(x, "%Y-%m"))
int_df.rename(columns={"DATE":"Time", "INTDSRINM193N":"Interest"}, inplace=True)
int_df

c:\users\najju\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\najju\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Time,Interest
0,2000-01,8.00
1,2000-02,8.00
2,2000-03,8.00
3,2000-04,7.00
4,2000-05,7.00
...,...,...
245,2020-06,4.25
246,2020-07,4.25
247,2020-08,4.25
248,2020-09,4.25


In [29]:
final = final.merge(int_df, on="Time")
final

,Time,Value,Exchange,Ratio_E_I,Interest
0,2000-01,2.619048,43.5895,69.665932,8.00
1,2000-02,3.614458,43.6540,73.462223,8.00
2,2000-03,4.830918,43.6383,70.899585,8.00
3,2000-04,5.542169,43.6847,75.393458,7.00
4,2000-05,5.011933,44.0759,86.969321,7.00
...,...,...,...,...,...
245,2020-06,5.063291,75.7077,105.855326,4.25
246,2020-07,5.329154,74.9286,86.198660,4.25
247,2020-08,5.625000,74.5657,78.095836,4.25
248,2020-09,5.636574,73.5233,87.053092,4.25


In [30]:
mon_df = pd.read_csv("money_m1_monthly.csv")
mon_df

,DATE,MANMM101INM189N
0,1960-01-01,2.609000e+10
1,1960-02-01,2.644000e+10
2,1960-03-01,2.720000e+10
3,1960-04-01,2.764000e+10
4,1960-05-01,2.745000e+10
...,...,...
725,2020-06-01,4.178770e+13
726,2020-07-01,4.264797e+13
727,2020-08-01,4.285648e+13
728,2020-09-01,4.324944e+13


In [31]:
mon_df["DATE"] = pd.to_datetime(mon_df["DATE"], format = "%Y-%m-%d")

In [32]:
datefilt4 = (mon_df["DATE"] >= "2000")
mon_df[datefilt4]

,DATE,MANMM101INM189N
480,2000-01-01,3.307250e+12
481,2000-02-01,3.372560e+12
482,2000-03-01,3.417960e+12
483,2000-04-01,3.509260e+12
484,2000-05-01,3.528510e+12
...,...,...
725,2020-06-01,4.178770e+13
726,2020-07-01,4.264797e+13
727,2020-08-01,4.285648e+13
728,2020-09-01,4.324944e+13


An extra modififcation we perform here is that the "MANMM101INM189N" column is converted from INR units to Billion INR units

In [33]:
mon_df = mon_df[datefilt4]
mon_df.set_index(pd.Index(newin), inplace=True)
mon_df["DATE"] = mon_df["DATE"].apply(lambda x: datetime.strftime(x, "%Y-%m"))
mon_df["MANMM101INM189N"] = mon_df["MANMM101INM189N"].apply(lambda x: x/(1e+12))
mon_df.rename(columns={"DATE":"Time", "MANMM101INM189N":"M1_supply"}, inplace=True)
mon_df

c:\users\najju\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\najju\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\najju\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4301: SettingWi

,Time,M1_supply
0,2000-01,3.30725
1,2000-02,3.37256
2,2000-03,3.41796
3,2000-04,3.50926
4,2000-05,3.52851
...,...,...
245,2020-06,41.78770
246,2020-07,42.64797
247,2020-08,42.85648
248,2020-09,43.24944


In [34]:
final = final.merge(mon_df, on="Time")
final

,Time,Value,Exchange,Ratio_E_I,Interest,M1_supply
0,2000-01,2.619048,43.5895,69.665932,8.00,3.30725
1,2000-02,3.614458,43.6540,73.462223,8.00,3.37256
2,2000-03,4.830918,43.6383,70.899585,8.00,3.41796
3,2000-04,5.542169,43.6847,75.393458,7.00,3.50926
4,2000-05,5.011933,44.0759,86.969321,7.00,3.52851
...,...,...,...,...,...,...
245,2020-06,5.063291,75.7077,105.855326,4.25,41.78770
246,2020-07,5.329154,74.9286,86.198660,4.25,42.64797
247,2020-08,5.625000,74.5657,78.095836,4.25,42.85648
248,2020-09,5.636574,73.5233,87.053092,4.25,43.24944


Now, the last data to be added to our dataset in the GDP data [6]. Since the data is produced quaterly, we need to modify it to monthly.

In [35]:
gdp_df = pd.read_csv("gdp_quaterly.csv")
gdp_df

,DATE,INDGDPNQDSMEI
0,01-04-96,3.340000e+12
1,01-07-96,3.420000e+12
2,01-10-96,3.540000e+12
3,01-01-97,3.620000e+12
4,01-04-97,3.710000e+12
...,...,...
94,01-10-19,5.130000e+13
95,01-01-20,5.220000e+13
96,01-04-20,3.840000e+13
97,01-07-20,3.430000e+13


In [36]:
gdp_df["DATE"] = pd.to_datetime(gdp_df["DATE"], format = "%d-%m-%y")

In [37]:
datefilt5 = (gdp_df["DATE"] >= "2000") & (gdp_df["DATE"] < "2021") #as 68 to 99 are interpreted as 2068 to 2099
gdp_df[datefilt5]

,DATE,INDGDPNQDSMEI
15,2000-01-01,5.140000e+12
16,2000-04-01,5.170000e+12
17,2000-07-01,5.280000e+12
18,2000-10-01,5.340000e+12
19,2001-01-01,5.430000e+12
...,...,...
94,2019-10-01,5.130000e+13
95,2020-01-01,5.220000e+13
96,2020-04-01,3.840000e+13
97,2020-07-01,3.430000e+13


In [38]:
gdp_df = gdp_df[datefilt5]
gdp_df["INDGDPNQDSMEI"] = gdp_df["INDGDPNQDSMEI"].apply(lambda x: x/(1e+12))
gdp_df.set_index(pd.Index(list(range(0,84))), inplace=True)
gdp_df

c:\users\najju\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,DATE,INDGDPNQDSMEI
0,2000-01-01,5.14
1,2000-04-01,5.17
2,2000-07-01,5.28
3,2000-10-01,5.34
4,2001-01-01,5.43
...,...,...
79,2019-10-01,51.30
80,2020-01-01,52.20
81,2020-04-01,38.40
82,2020-07-01,34.30


We will create a new dataframe to store the monthly GDP values

In [39]:
date_range = [d.strftime("%Y-%m") for d in pd.date_range(start="2000-01", end="2020-11", freq="M")]
temp = {"Time":date_range, "GDP":list(range(0,250))}
gdp_df2 = pd.DataFrame(temp)
gdp_df2

,Time,GDP
0,2000-01,0
1,2000-02,1
2,2000-03,2
3,2000-04,3
4,2000-05,4
...,...,...
245,2020-06,245
246,2020-07,246
247,2020-08,247
248,2020-09,248


In [40]:
gdp_df2["Time"] = pd.to_datetime(gdp_df2["Time"], format = "%Y-%m")
gdp_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Time    250 non-null    datetime64[ns]
 1   GDP     250 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 4.0 KB


The function below is designed to repeat out quaterly data for the respective 4 months that it belongs to according to the dataset description.

In [41]:
for i in range(0,250):
    month = gdp_df2.loc[i,"Time"].month
    year = gdp_df2.loc[i,"Time"].year
    
    if (month in [1,2,3]):
        valfilt = (gdp_df["DATE"].dt.year == year) & (gdp_df["DATE"].dt.month == 1)
        gdp_df2.loc[i:i+2, "GDP"] = gdp_df[valfilt]["INDGDPNQDSMEI"].values[0]
        
    if (month in [4,5,6]):
        valfilt = (gdp_df["DATE"].dt.year == year) & (gdp_df["DATE"].dt.month == 4)
        gdp_df2.loc[i:i+2, "GDP"] = gdp_df[valfilt]["INDGDPNQDSMEI"].values[0]
        
    if (month in [7,8,9]):
        valfilt = (gdp_df["DATE"].dt.year == year) & (gdp_df["DATE"].dt.month == 7)
        gdp_df2.loc[i:i+2, "GDP"] = gdp_df[valfilt]["INDGDPNQDSMEI"].values[0]
        
    if (month in [10,11,12]):
        valfilt = (gdp_df["DATE"].dt.year == year) & (gdp_df["DATE"].dt.month == 10)
        gdp_df2.loc[i:i+2, "GDP"] = gdp_df[valfilt]["INDGDPNQDSMEI"].values[0]
        
        
gdp_df2

,Time,GDP
0,2000-01-01,5.14
1,2000-02-01,5.14
2,2000-03-01,5.14
3,2000-04-01,5.17
4,2000-05-01,5.17
...,...,...
245,2020-06-01,38.40
246,2020-07-01,34.30
247,2020-08-01,34.30
248,2020-09-01,34.30


We perform the final merge and view are dataframe.

In [46]:
final = final.merge(gdp_df2, on="Time")
final

,Time,Value,Exchange,Ratio_E_I,Interest,M1_supply,GDP
0,2000-01-01,2.619048,43.5895,69.665932,8.00,3.30725,5.14
1,2000-02-01,3.614458,43.6540,73.462223,8.00,3.37256,5.14
2,2000-03-01,4.830918,43.6383,70.899585,8.00,3.41796,5.14
3,2000-04-01,5.542169,43.6847,75.393458,7.00,3.50926,5.17
4,2000-05-01,5.011933,44.0759,86.969321,7.00,3.52851,5.17
...,...,...,...,...,...,...,...
245,2020-06-01,5.063291,75.7077,105.855326,4.25,41.78770,38.40
246,2020-07-01,5.329154,74.9286,86.198660,4.25,42.64797,34.30
247,2020-08-01,5.625000,74.5657,78.095836,4.25,42.85648,34.30
248,2020-09-01,5.636574,73.5233,87.053092,4.25,43.24944,34.30


The last step is to save the dataframe as a CSV file to be imported and modelled in R later.

In [47]:
final.to_csv("total_data.csv", index=False)

#### References:
[1] OECD (2021), Inflation (CPI) (indicator). doi: 10.1787/eee82e6e-en (Accessed on 19 December 2020)  
[2] Board of Governors of the Federal Reserve System (US), India / U.S. Foreign Exchange Rate [EXINUS], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/EXINUS, December 18, 2020.  
[3] Organization for Economic Co-operation and Development, Ratio of Exports to Imports for India [XTEITT01INM156S], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/XTEITT01INM156S, December 19, 2020.  
[4] International Monetary Fund, Interest Rates, Discount Rate for India [INTDSRINM193N], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/INTDSRINM193N, December 19, 2020.  
[5] Organization for Economic Co-operation and Development, M1 for India [MANMM101INM189N], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/MANMM101INM189N, December 19, 2020.  
[6] Organization for Economic Co-operation and Development, Constant Price Gross Domestic Product in India [INDGDPRQPSMEI], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/INDGDPRQPSMEI, December 19, 2020.  